In [26]:
import sys, os
sys.path.append('..')

from utils.prompts import render
from utils.llm_client import LLMClient # Pick llm client
from utils.logging_utils import log_llm_call # logging every API call
from utils.router import pick_model, should_use_reasoning_model # # general, resooning or strong
from IPython.display import Markdown


## 01 - Zero Shot

In [6]:
prompt_text, spec = render(
            "zero_shot.v1",
            role="sentiment_analyst",
            instruction="Analyze the following text and determine the sentiment as positive / negative neutral.",
            constraints="The sentiment should be one of the following: positive, negative, neutral",
            format="The sentiment is: {sentiment}"
            )

model = pick_model('openai', 'general')
llm = LLMClient('openai', model)

text = "I'am really happy with the product! It's amazing!"
messages= [
    {
        "role": "user",
        "content": f"{prompt_text}\n\nReview: {text}"
    }]

llm.chat(messages, temperature=0.0)




{'text': 'The sentiment is: positive',
 'usage': {'input_tokens_est': 73,
  'context_tokens_est': 0,
  'total_est': 76,
  'prompt_tokens_actual': 76,
  'completion_tokens_actual': 5,
  'total_tokens_actual': 81},
 'latency_ms': 923,
 'raw': ChatCompletion(id='chatcmpl-Cs62MWy4tWFwRYoBVG4hF1E0LOWOJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The sentiment is: positive', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1767008954, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_8bbc38b4db', usage=CompletionUsage(completion_tokens=5, prompt_tokens=76, total_tokens=81, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))),
 'meta': {'retry_count': 0, 

## 02 - Few Shot

In [ ]:
examples = """
Example 01:
Review: I'm really happy with the product! It's amazing!
Sentiment: positive
Explanation: User says product is good also he is happy.

Example 02:
Review: I'm really unhappy with the product! It's bad!
Sentiment: negative
Explanation: User says product is bad and he is unhappy.

Example 03:
Review: The product is okay, but it's not great.
Sentiment: neutral
Explanation: User says product is okay but not great.

Example 04:
Review: I'm not sure about the product.
Sentiment: neutral
Explanation: User not sure about the product.

Example 05:
Review: I'm really happy with the product but It's bad!
Sentiment: negative
Explanation: User says product is good but he is happy. So we are prioratizing his word on product quality not his happiness.
             That's why we are considering thee product quality as negative.

Example 06:
Review: I'm really unhappy with the product but It's amazing!
Sentiment: positive
Explanation: User says product is bad but he is happy. So we are prioratize his word on product quality not on his happiness.
             That's why we are considering thee product quality as positive.


"""

prompt_text, spec = render(
            "zero_shot.v1",
            role="sentiment_analyst",
            examples=examples,
            instruction="Analyze the following text and determine the sentiment as positive / negative neutral.",
            constraints="The sentiment should be one of the following: positive, negative, neutral",
            format="The sentiment is: {sentiment}"
            )

model = pick_model('groq', 'reason')
llm = LLMClient('groq', model)

text = "It seems nice but It's."
messages= [
    {
        "role": "user",
        "content": f"{prompt_text}\n\nReview: {text}"
    }]

response = llm.chat(messages, temperature=0.4)
print(response['text'])

The sentiment is: negative


## 03 -  COT

In [34]:
model = pick_model('groq', 'reason')
llm = LLMClient('groq', model)

problem = """
A car travels 100 miles in 2 hours. What is the average speed of the car?
also if car stops for 40 minutes what is the average speed of the car?
"""


prompt_text, spec = render(
            "cot_reasoning.v1",
            role="math_tutor",
            instruction="""slove the following problem step by step. 
            1. First identify whether car travelled the entire time without stopping or not.
            2. If car stopped for x minutes and overall travelled for y the travel duration is y-x. So the speed should be d / (y-x).
            3. If stopping time x mentioned do not add it to the travel duration. because it's already included in the travel duration of 2 hours.
            So actual trael time is y (total travel time) - x (stopping time)
            4. If car travelled the entire time without stopping then the average speed is d / y.
            5. If car stopped for x minutes then the average speed is d / (y - x) 
            """,
            problem=problem
            )

messages= [
    {
        "role": "user",
        "content": f"{prompt_text}"
    }]

response = llm.chat(messages, temperature=0.0)
display(Markdown(response['text']))

**Reasoning steps**

1. **Average speed** is defined as total distance divided by total time (including any stops).  
2. Without any stop:  
   - Distance = 100 mi  
   - Time = 2 h  
   - Speed = 100 mi / 2 h = 50 mph.  
3. With a 40‑minute stop:  
   - Convert the stop time to hours: 40 min = 40/60 h = 2/3 h ≈ 0.6667 h.  
   - Total elapsed time = 2 h + 2/3 h = 8/3 h ≈ 2.6667 h.  
   - Average speed = 100 mi / (8/3 h) = 100 × 3/8 = 300/8 = 37.5 mph.

**Answer**

- Average speed without stopping: **50 mph**.  
- Average speed when the car stops for 40 minutes: **37.5 mph**.

## 04 - COT